### Possible libraries and models to use: flair, fastai, allennlp.

### Summary report: https://nlpprogress.com/

# PyTorch Official Tutorial

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [5]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

In [6]:
inputs

[tensor([[-0.5525,  0.6355, -0.3968]]),
 tensor([[-0.6571, -1.6428,  0.9803]]),
 tensor([[-0.0421, -0.8206,  0.3133]]),
 tensor([[-1.1352,  0.3773, -0.2824]]),
 tensor([[-2.5667, -1.4303,  0.5009]])]

In [4]:
# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))

In [5]:
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

In [7]:
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.2682,  0.0304, -0.1526]],

        [[-0.5370,  0.0346, -0.1958]],

        [[-0.3947,  0.0391, -0.1217]],

        [[-0.1854,  0.0740, -0.0979]],

        [[-0.3600,  0.0893,  0.0215]]], grad_fn=<StackBackward>)
(tensor([[[-0.3600,  0.0893,  0.0215]]], grad_fn=<StackBackward>), tensor([[[-1.1298,  0.4467,  0.0254]]], grad_fn=<StackBackward>))


In [8]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [9]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [10]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-1.0388, -0.9874, -1.2962],
        [-0.9900, -1.0083, -1.3334],
        [-0.9792, -1.0514, -1.2912],
        [-0.9556, -1.0550, -1.3197],
        [-0.9470, -1.0579, -1.3284]])
tensor([[-0.0882, -2.5320, -5.3084],
        [-3.8870, -0.0483, -3.6236],
        [-2.2643, -3.0358, -0.1648],
        [-0.0950, -2.6379, -3.9579],
        [-4.1251, -0.0204, -5.5152]])


# Flair (https://github.com/zalandoresearch/flair)


In [13]:
from flair.data import Sentence
from flair.models import SequenceTagger

# make a sentence
sentence = Sentence('I love Berlin .')

# load the NER tagger
tagger = SequenceTagger.load('ner')

# run NER over sentence
tagger.predict(sentence)


2019-02-19 14:27:09,622 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.2/NER-conll03--h256-l1-b32-%2Bglove%2Bnews-forward%2Bnews-backward--v0.2/en-ner-conll03-v0.2.pt not found in cache, downloading to /tmp/tmpboqnz1od


100%|██████████| 432921440/432921440 [01:46<00:00, 4048794.15B/s]

2019-02-19 14:28:57,166 copying /tmp/tmpboqnz1od to cache at /home/snie/.flair/models/en-ner-conll03-v0.2.pt


2019-02-19 14:28:57,417 removing temp file /tmp/tmpboqnz1od


[Sentence: "I love Berlin ." - 4 Tokens]

In [16]:
# load the POS tagger
pos_tagger = SequenceTagger.load('pos')

2019-02-19 14:32:43,955 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.2/POS-ontonotes--h256-l1-b32-%2Bmix-forward%2Bmix-backward--v0.2/en-pos-ontonotes-v0.2.pt not found in cache, downloading to /tmp/tmpgdgewkth


100%|██████████| 248585588/248585588 [01:04<00:00, 3852261.29B/s]

2019-02-19 14:33:49,048 copying /tmp/tmpgdgewkth to cache at /home/snie/.flair/models/en-pos-ontonotes-v0.2.pt


2019-02-19 14:33:49,198 removing temp file /tmp/tmpgdgewkth


In [17]:
pos_tagger.predict(sentence)

[Sentence: "I love Berlin ." - 4 Tokens]

In [20]:
sentence.to_tagged_string()

'I <PRP> love <VBP> Berlin <S-LOC/NNP> . <.>'

In [59]:
pos_tagger.parameters

<bound method Module.parameters of SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): CharLMEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
    (list_embedding_1): CharLMEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
  )
  (embedding2nn): Linear(in_features=4096, out_features=4096, bias=True)
  (rnn): LSTM(4096, 256, bidirectional=True)
  (linear): Linear(in_features=512, out_features=53, bias=True)
)>

In [60]:
t = pos_tagger.rnn